In [89]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

data = pd.read_csv('./src/USA_cars_datasets.csv')

def calc_corr(x, y):
    print(abs(np.corrcoef(x, y)[0, 1]))
    
def replace_string_values(data, sorted_unique_values):
    i = 0
    values = {}

    for value in sorted_unique_values:
        if value in values:
            continue

        values[value] = i

        i += 1

    return np.array(list(map(lambda x : values[x], data)))

def sorted_by_avg_price(values):
    x_values = np.unique(values)  
    y_values = []

    for x in x_values:
        y_values.append(data.price.values[values == x].mean())

    df = pd.DataFrame({
        'x_values': x_values,
        'y_values': y_values
    }).sort_values(by=['y_values'])

    return list(df.x_values)

data.head()

,Unnamed: 0,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition
0,0,6300,toyota,cruiser,2008,clean vehicle,274117.0,black,jtezu11f88k007763,159348797,new jersey,usa,10 days left
1,1,2899,ford,se,2011,clean vehicle,190552.0,silver,2fmdk3gc4bbb02217,166951262,tennessee,usa,6 days left
2,2,5350,dodge,mpv,2018,clean vehicle,39590.0,silver,3c4pdcgg5jt346413,167655728,georgia,usa,2 days left
3,3,25000,ford,door,2014,clean vehicle,64146.0,blue,1ftfw1et4efc23745,167753855,virginia,usa,22 hours left
4,4,27700,chevrolet,1500,2018,clean vehicle,6654.0,red,3gcpcrec2jg473991,167763266,florida,usa,22 hours left


В данном разделе проверим признаки нашего датасета на пригодность. В качестве зависимой переменной возьмем цену автомобиля. Будем проверять корелляцию независимых переменных. Самыми логичными признаками для прогнозирования цены авто кажутся 2 признака: пробег и год выпуска; Убедимся в этом.

In [72]:
x = data.year.values
y = data.price.values

calc_corr(x, y)

x = data.mileage.values
y = data.price.values

calc_corr(x, y)

0.4182735684917936
0.400838186329367


Несмотря на то, что корелляция далека от единицы, показатели подтверждают нашу гипотезу о пригодности признаков. Также стоит отметить, что признаки почти равноценны.

Также в датасете присутствует серийный номер, он разбит на 2 столбца - vin и lot (вин номер авто и номер партии). Логично, что от этой парочки значений мало толку в прогнозировании цены авто, так что долго останавливаться на них не будем, лишь убедимся, что их корелляция слаба.

In [73]:
x = data.lot.values
y = data.price.values

calc_corr(x, y)

0.15936924684631346


В датасете имеется статус автомобиля, говорящий нам о том, застраховано ли авто.  В датасете это бинарный фактор, но записаный строковым значением, переведем это в (0, 1). Посмотрим, влияет ли этот фактор на зависимую переменную.

In [86]:
x = replace_string_values(data.title_status.values, np.unique(data.title_status.values))
y = data.price.values

calc_corr(x, y)

0.3583789487651185


В целом, да, влияет.

Теперь перейдем к марке авто. Для подсчета корелляции необходимо перевести строковые значения в численные данные. Сделаем это для начала на "абум", пронумеровав марки авто по алфавиту.

In [85]:
x = replace_string_values(data.brand.values, np.unique(data.brand.values))
y = data.price.values

calc_corr(x, y)

0.21573307062467076


Значение корелляции слабое, попробуем его улучшить путем правильного индексирования марок авто. Отсортируем бренды по средней цене автомобиля.

In [91]:
x = replace_string_values(data.brand.values, sorted_by_avg_price(data.brand.values))
y = data.price.values

calc_corr(x, y)

0.33887165017681614


Получилось улучшить результат, но всё равно недостаточно хорошо. Попробуем поколдовать с моделью машины. Отдельно от марки нет смысла рассматривать отдельные модели, если модели могут совпадать среди разных марок(Как, например, скитание модели Lanos среди бреднов Daewoo, Shevrolet, ЗАЗ). Если это так, то необходимо объединять признаки бренд и модель в один. Но для начала проверим, совпадают ли модели среди марок авто? 

In [102]:
df = pd.DataFrame(data, columns=["model", "brand"])

brands_by_model = {}

models = df.model.values
brands = df.brand.values

for i in range(len(models)):
    if models[i] in brands_by_model and brands[i] in brands_by_model[models[i]]:
        continue
    
    brands_by_model[m[i]] = [brands[i]]
    
for model in brands_by_model:
    if len(brands_by_model[model]) > 1:
        print(model, brands_by_model[model])

Отлично, модели авто уникальны среди всех брендов! Тогда проверим на пригодность признак модели авто.

In [103]:
x = replace_string_values(data.model.values, sorted_by_avg_price(data.model.values))
y = data.price.values

calc_corr(x, y)

0.7346570048904479


Отлично! Самая лучшая корелляция у признака модель машины, если нумеровать название модели по порядку увеличения/уменьшения средней стоимости авто. Таким образом, думаю, что стоит отобрать 3 следующих признака в качестве независимых переменных: пробег автомобиля, модель автомобиля и год его выпуска.
